In [3]:

from scipy.io import arff
import numpy as np
import json
from sklearn.model_selection import train_test_split, KFold

dataset = arff.loadarff(open('dataset.arff', 'r'))

data = np.array([np.array(list(ele), dtype='int') for ele in dataset[0]])

print('_______')


print('Датасет имеет {0} записей, и {1} фичей'.format(data.shape[0], data.shape[1]-1))
print('Фичи: {0}'.format([feature for feature in dataset[1]]))


print([feature for feature in dataset[1]].index('port'))
data = data[:, [0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 16, 22, 30]]

X, y = data[:, :-1], data[:, -1]
y.reshape(y.shape[0])

print('Размеры до деления на тест и трэин')
print('X:{0}, y:{1}'.format(X.shape, y.shape))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print('После деления')
print('X_train:{0}, y_train:{1}, X_test:{2}, y_test:{3}'.format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)



_______
Датасет имеет 11055 записей, и 30 фичей
Фичи: ['having_IP_Address', 'URL_Length', 'Shortining_Service', 'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix', 'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length', 'Favicon', 'port', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor', 'Links_in_tags', 'SFH', 'Submitting_to_email', 'Abnormal_URL', 'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe', 'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank', 'Google_Index', 'Links_pointing_to_page', 'Statistical_report', 'Result']
10
Размеры до деления на тест и трэин
X:(11055, 16), y:(11055,)
После деления
X_train:(7738, 16), y_train:(7738,), X_test:(3317, 16), y_test:(3317,)


In [4]:
N = 16
from sklearn.tree import _tree


def tree_to_json(tree):
    tree_ = tree.tree_
    feature_names = range(30)
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    def recurse(node):
        tree_json = dict()
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            tree_json['type'] = 'split'
            threshold = tree_.threshold[node]
            tree_json['threshold'] = "{} <= {}".format(feature_name[node], threshold)
            tree_json['left'] = recurse(tree_.children_left[node])
            tree_json['right'] = recurse(tree_.children_right[node])
        else:
            tree_json['type'] = 'leaf'
            tree_json['value'] = tree_.value[node].tolist()
        return tree_json

    return recurse(0)



def forest_to_json(forest):
    forest_json = dict()
    forest_json['n_features'] = N
    forest_json['n_classes'] = forest.n_classes_
    forest_json['classes'] = forest.classes_.tolist()
    forest_json['n_outputs'] = forest.n_outputs_
    forest_json['n_estimators'] = forest.n_estimators
    forest_json['estimators'] = [tree_to_json(estimator) for estimator in forest.estimators_]
    return forest_json



In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
import numpy as np
import json


X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
print('X_train:{0}, y_train:{1}'.format(X_train.shape, y_train.shape))

clf = RandomForestClassifier()

clf.fit(X_train, y_train)

X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')


pred = clf.predict(X_test)
print('Точность {}'.format(accuracy_score(y_test, pred)))


json.dump(forest_to_json(clf), open('classifier.json', 'w'))



X_train:(7738, 16), y_train:(7738,)
Точность 0.9463370515526078
